In [ ]:
import pandas as pd
import numpy as np
import json
import plotly.express as px

import plotly.io as pio
from plotly import graph_objects

In [ ]:

data = pd.read_csv('economic_freedom_index2019_data.csv', encoding='latin-1')
null_c = data[data['World Rank'].isnull()] ## countries w nan values, need to be removed
data = data[data['World Rank'].notna()] #### remove those countries
no = data.columns[data.isna().any()].tolist() ### columns w nan values, need to be removed
data = data.drop(no,axis=1) #### remove those columns
data = data.drop(['Country'],axis=1) ### Drop column b/c it messes w loop

columns = list(data.columns.values) 

# ##############################################################

data = data.replace(',','', regex=True)
data[columns[22]] = data[columns[22]].str.replace('$', '')
data[columns[24]] = data[columns[24]].str.replace('$', '')
data.loc[88,[columns[22]]] = 40 ###hardcode
data.loc[88,[columns[24]]] = 1700 ##hardcode

dataECON = data
dataHD = pd.read_csv('human_development.csv', encoding='latin-1')
dataHD = dataHD.iloc[:188,[i for i in range(8) ] ]
null_data = dataHD[dataHD.isnull().any(axis=1)]
dataEDU = dataHD.loc[:,['Country','Mean Years of Education']]
#dataEDU = dataHD.iloc[:188,[i for i in range(8) ] ]
dataEDU=  dataEDU.set_index(['Country'])
dataECON = dataECON.set_index(['Country Name'])
dataECON = dataECON['World Rank']
df = pd.merge(dataECON,dataEDU,left_index=True, right_index=True)


In [ ]:
map3 = json.load(open('countries.geojson','r'))
for i in range(255):
    (map3['features'][i]['id']) = i

id_map = {}
for feature in map3["features"]:

    id_map[feature["properties"]["ADMIN"]] = feature["id"]




df  = df.reset_index() 
#df.at[151,'index']='United States of America'
s = df['index'].tolist()

store1 =[]

for key in id_map:  ## check to see if in original df
    b = key in s
    if b == False:
        store1.append(key)

s = df['index'].tolist()
for s in store1: 
    del id_map[s]

s = df['index'].tolist()   
for i in s:
    if i not in id_map:
        df = df[df['index'] != i]
      
df["id"] = df["index"].apply(lambda x: id_map[x])


In [ ]:
import plotly.graph_objects as go

g  = ['World Rank','Mean Years of Education']
for i in range(2):
    fig = go.Figure(go.Choroplethmapbox(geojson=map3, locations=df.id, z=df[g[i]],
                                        colorscale="Viridis", zmin=0, zmax= np.max((df['World Rank'].to_numpy())),
                                        marker_opacity=0.5, marker_line_width=0))
    fig.update_layout(mapbox_style="carto-positron",
                      mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()
    
    updatemenus = list([dict(buttons=list()), 
                        dict(direction='down',
                             showactive=True)])
